<a href="https://colab.research.google.com/github/Vincethevince/CaCapstone/blob/init-colab/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract

With this project, I want to summarize what I learned in the Codecademy course. Even though we could simply utilize LLMs for a chatbot, I want to create one on my own. I aim to test different inputs and observe how they affect the performance of the chatbot itself.

The different inputs are


*   a human chat with ~2k lines
*   dialogs from a random dataset ~3k lines
*   dialogs from movies 200k+ lines

Outputs
*   are only "hi" and "i am <UNK> <UNK>"
*   i am not, i am going to, i am ...


In [9]:
# Needed for movie corpus
!pip install convokit

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2


## Imports

In [2]:
import numpy as np
import re
import unicodedata
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.callbacks import EarlyStopping
#from convokit import Corpus, download
from collections import Counter

## Preprocessing

In [3]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def clean_text(text):
    text = unicode_to_ascii(text.lower().strip())
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r'human (\d): ','',text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub("(\\W)"," ",text)
    text = re.sub('\S*\d\S*\s*','', text)
    return text


In [4]:
wordCount = Counter()

# Function to update word count
def update_word_count(docs):
    for line in docs:
        wordCount.update(line.split())

### Movies

In [ ]:
corpus = Corpus(filename=download("movie-corpus"))
corpus

In [11]:
import urllib
import jupyter
from google.colab import files
uploaded = files.upload()
import io
filename = list(uploaded.keys())[0]
with io.open(filename, 'r', encoding='utf-8') as f:
    content = f.read()
print(content)

conv_path = "https://raw.githubusercontent.com/Vincethevince/CaCapstone/init-colab/movie_conversations.txt/movie_conversations.txt"
lines_path = "https://raw.githubusercontent.com/Vincethevince/CaCapstone/init-colab/movie_conversations.txt/movie_lines_small.txt"

try:
  convers = urllib.request.urlopen(conv_path).read()
  convers = convers.decode("utf-8").split("\n")
  print(convers)
  lines = urllib.request.urlopen(lines_path).read()
  lines = lines.decode("utf-8").split("\n")
except:
  print("Problem with downloading files")

exchn = []
for conver in convers:
    exchn.append(conver.split(' +++$+++ ')[-1][1:-1].replace("'", " ").replace(",","").split())

diag = {}
for line in lines:
    diag[line.split(' +++$+++ ')[0]] = line.split(' +++$+++ ')[-1]

input_docs = []
target_docs = []

for conver in exchn:
    for i in range(len(conver) - 1):
      if conver[i] in diag and conver[i+1] in diag:
        input_docs.append(diag[conver[i]])
        target_docs.append(diag[conver[i+1]])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Problem with downloading files


AttributeError: 'int' object has no attribute 'split'

### Dialogs

In [38]:
import urllib
data_path = "https://raw.githubusercontent.com/Vincethevince/CaCapstone/init-colab/dialogs.txt"
try:
    lines = urllib.request.urlopen(data_path).read()
    lines = lines.decode("utf-8").split("\n")
except:
    print(f"Error: File {data_path} not found.")
    lines = []

# Ensure there are enough lines to form pairs
if len(lines) < 2:
    print("Not enough lines to form pairs.")
    pairs = []
else:
  pairs = [[text for text in line.strip().split("\t")] for line in lines]
  print(pairs)

[['hi, how are you doing?', "i'm fine. how about yourself?"], ["i'm fine. how about yourself?", "i'm pretty good. thanks for asking."], ["i'm pretty good. thanks for asking.", 'no problem. so how have you been?'], ['no problem. so how have you been?', "i've been great. what about you?"], ["i've been great. what about you?", "i've been good. i'm in school right now."], ["i've been good. i'm in school right now.", 'what school do you go to?'], ['what school do you go to?', 'i go to pcc.'], ['i go to pcc.', 'do you like it there?'], ['do you like it there?', "it's okay. it's a really big campus."], ["it's okay. it's a really big campus.", 'good luck with school.'], ['good luck with school.', 'thank you very much.'], ["how's it going?", "i'm doing well. how about you?"], ["i'm doing well. how about you?", 'never better, thanks.'], ['never better, thanks.', 'so how have you been lately?'], ['so how have you been lately?', "i've actually been pretty good. you?"], ["i've actually been pretty 

### Human_chat

In [21]:
import urllib

# Try reading txt file from github
data_path = "https://raw.githubusercontent.com/Vincethevince/CaCapstone/init-colab/human_chat.txt"
try:
    lines = urllib.request.urlopen(data_path).read()
    lines = lines.decode("utf-8").split("\n")
except:
    print(f"Error: File {data_path} not found.")
    lines = []

# Ensure there are enough lines to form pairs
if len(lines) < 2:
    print("Not enough lines to form pairs.")
    pairs = []
else:
  # Group lines by pairs and exclude overlapping conversations
  pairs = [(lines[i], lines[i+1]) for i in range(len(lines)-1) if "human 1: hi" not in lines[i+1]]
  print(pairs)

[('Human 1: Hi!', 'Human 2: What is your favorite holiday?'), ('Human 2: What is your favorite holiday?', 'Human 1: one where I get to meet lots of different people.'), ('Human 1: one where I get to meet lots of different people.', 'Human 2: What was the most number of people you have ever met during a holiday?'), ('Human 2: What was the most number of people you have ever met during a holiday?', 'Human 1: Hard to keep a count. Maybe 25.'), ('Human 1: Hard to keep a count. Maybe 25.', 'Human 2: Which holiday was that?'), ('Human 2: Which holiday was that?', 'Human 1: I think it was Australia'), ('Human 1: I think it was Australia', 'Human 2: Do you still talk to the people you met?'), ('Human 2: Do you still talk to the people you met?', "Human 1: Not really. The interactions are usually short-lived but it's fascinating to learn where people are coming from and what matters to them"), ("Human 1: Not really. The interactions are usually short-lived but it's fascinating to learn where pe

### General

In [39]:
input_docs = []
target_docs = []

max_len = 15

for line in pairs:
  input_doc, target_doc = line[0], line[1]
  input_doc = clean_text(input_doc)
  input_doc = " ". join(word for word in input_doc.split()[:max_len])

  input_docs.append(input_doc)

  target_doc = clean_text(target_doc)
  target_doc = '<START> ' + target_doc + ' <END>'
  target_doc = " ". join(word for word in target_doc.split()[:max_len])
  target_docs.append(target_doc)


update_word_count(input_docs)
update_word_count(target_docs)

In [40]:
# Filter barely used words out and add special tokens
threshhold = 4
spec_tokens = ["<PAD>","<UNK>"]
vocab = [key for key, value in wordCount.items() if value > threshhold] + spec_tokens
vocab = sorted(vocab)

# Create vocab dict and reverse dict
vocab_dict = {}
word_num = 0
for word in vocab:
  vocab_dict[word] = word_num
  word_num += 1

inv_vocab = {v:k for k,v in vocab_dict.items()}

In [41]:
def encode_sentences(docs, vocab):
  encoded_sentences = []
  for line in docs:
    encoded_line = [vocab.get(word, vocab['<UNK>']) for word in line.split()]
    encoded_sentences.append(encoded_line)
  return encoded_sentences

encoder_input = encode_sentences(input_docs, vocab_dict)
decoder_input = encode_sentences(target_docs, vocab_dict)

# Pad the inputs
encoder_input_data = pad_sequences(encoder_input, maxlen=max_len, padding="post")
decoder_input_data = pad_sequences(decoder_input, maxlen=max_len, padding="post")

# Create target data shifted by one timestep
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

# One-hot encode the target data
num_tokens = len(vocab_dict) + 1


decoder_input_data = to_categorical(decoder_input_data, num_classes=num_tokens)
decoder_target_data = to_categorical(decoder_target_data, num_classes=num_tokens)

## Train model

In [42]:
dimensionality = 256
# Hyperparameters
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(input_dim=num_tokens, output_dim=latent_dim, input_length=max_len)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=100,
          validation_split=0.2, callbacks=[early_stopping])


encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

Epoch 1/100
47/47 [==============================] - 26s 474ms/step - loss: 3.7517 - accuracy: 0.5433 - val_loss: 3.0468 - val_accuracy: 0.5325
Epoch 2/100
47/47 [==============================] - 19s 416ms/step - loss: 2.7165 - accuracy: 0.5691 - val_loss: 3.0202 - val_accuracy: 0.5327
Epoch 3/100
47/47 [==============================] - 20s 424ms/step - loss: 2.6698 - accuracy: 0.5703 - val_loss: 2.9911 - val_accuracy: 0.5333
Epoch 4/100
47/47 [==============================] - 18s 386ms/step - loss: 2.6242 - accuracy: 0.5728 - val_loss: 2.9479 - val_accuracy: 0.5407
Epoch 5/100
47/47 [==============================] - 18s 387ms/step - loss: 2.5692 - accuracy: 0.5747 - val_loss: 2.9219 - val_accuracy: 0.5409
Epoch 6/100
47/47 [==============================] - 19s 407ms/step - loss: 2.5181 - accuracy: 0.5772 - val_loss: 2.8683 - val_accuracy: 0.5417
Epoch 7/100
47/47 [==============================] - 19s 394ms/step - loss: 2.4763 - accuracy: 0.5773 - val_loss: 2.8534 - val_accuracy:

## Chat

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_tokens))
    # Populate the first token of target sequence with the start token.
    target_seq[0, 0, vocab_dict['<START>']] = 1.

    # Sampling loop for a batch of sequences
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = inv_vocab[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length or find stop character.
        if sampled_char == '<END>' or len(decoded_sentence) > max_len:
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

def string_to_matrix(user_input):
    tokens = re.findall(r"<?[\w']+>?|<?[^\s\w]>?", user_input.lower())
    user_input_matrix = np.zeros((1, max_len), dtype='int32')

    for timestep, token in enumerate(tokens[:max_len]):
        if token in vocab_dict:
            user_input_matrix[0, timestep] = vocab_dict[token]
        else:
            user_input_matrix[0, timestep] = vocab_dict["<UNK>"]

    return user_input_matrix

# Example usage
test_input = "Hi how are you"
while test_input!= "exit":
    test_input_matrix = string_to_matrix(test_input)
    test_input = input(decode_sequence(test_input_matrix))


1/1 [==============================] - 0s 35ms/step
 i am not you <END>who are you
1/1 [==============================] - 0s 24ms/step
 i am not you wantdo you have a plan for tomorrow
1/1 [==============================] - 0s 22ms/step
 i am not you likei like pizza what about you
1/1 [==============================] - 0s 30ms/step
 i am not you <END>can we please talk like two normal beings 
1/1 [==============================] - 0s 35ms/step
